<a href="https://colab.research.google.com/github/didulabhanuka/Tomato-Ripeness-Classifier/blob/main/Model_evaluations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ultralytics torch torchvision torchmetrics


In [ ]:
import os
import cv2
import json
import yaml
import torch
import random
import shutil
import zipfile
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import functional as F
from matplotlib.backends.backend_pdf import PdfPages
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from sklearn.metrics import precision_recall_curve, average_precision_score, precision_recall_fscore_support
from albumentations import Compose, RandomBrightnessContrast, HueSaturationValue, GaussianBlur, MotionBlur, Normalize


from google.colab import drive
drive.mount('/content/drive')


In [ ]:

yaml_content = """
train: /content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/train
val: /content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val

nc: 6
names: ["b_fully_ripened", "b_half_ripened", "b_green", "l_fully_ripened", "l_half_ripened", "l_green"]
"""

yaml_path = "/content/tomato_ripeness_classifier.yaml"
with open(yaml_path, "w") as file:
    file.write(yaml_content)

print(f"YAML file created at {yaml_path}")

In [ ]:
dataset_zip = "/content/drive/MyDrive/tomato_ripeness_classifier/tomato_dataset.zip"
dataset_dir = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset/dataset"

os.makedirs("/content/dataset", exist_ok=True)
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")

print("Dataset unzipped successfully.")

In [ ]:
# Define dataset paths
augmented_dir = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented"

# Create Augmented Dataset Folders
os.makedirs(f"{augmented_dir}/train/images", exist_ok=True)
os.makedirs(f"{augmented_dir}/train/labels", exist_ok=True)
os.makedirs(f"{augmented_dir}/val/images", exist_ok=True)
os.makedirs(f"{augmented_dir}/val/labels", exist_ok=True)

def denormalize_image(image):
    """
    Reverts normalization to bring pixel values back to [0,255].
    """
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    # Undo normalization
    image = image * std + mean  # Reverse normalization
    image = np.clip(image * 255, 0, 255).astype(np.uint8)  # Convert to 0-255 range

    return image

def validate_and_clip_bbox(bbox, img_w, img_h):
    """
    Ensures bounding box values stay within valid ranges.
    """
    x_center, y_center, width, height = bbox
    x_center /= img_w
    y_center /= img_h
    width /= img_w
    height /= img_h

    x_center = np.clip(x_center, 0.0, 1.0)
    y_center = np.clip(y_center, 0.0, 1.0)
    width = np.clip(width, 0.0, 1.0)
    height = np.clip(height, 0.0, 1.0)

    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    if 0.0 <= x_min <= 1.0 and 0.0 <= y_min <= 1.0 and 0.0 <= x_max <= 1.0 and 0.0 <= y_max <= 1.0 and width > 0 and height > 0:
        return [x_center, y_center, width, height]
    return None  # Invalid bbox

def advanced_augmentations(image_folder, label_folder, output_image_folder, output_label_folder):
    """
    Applies augmentations while keeping bounding boxes correctly aligned.
    """
    augmentations = Compose(
        [
            RandomBrightnessContrast(p=0.2),
            HueSaturationValue(p=0.2),
            GaussianBlur(p=0.1),
            MotionBlur(p=0.1),
            Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # Applied for training
        ],
        bbox_params={"format": "yolo", "label_fields": ["class_labels"]},
    )

    for image_file in os.listdir(image_folder):
        img_path = os.path.join(image_folder, image_file)
        label_path = os.path.join(label_folder, os.path.splitext(image_file)[0] + ".txt")

        # Read the image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Skipping {image_file}: Unable to read image.")
            continue

        h, w, _ = image.shape
        bboxes = []
        class_labels = []

        # Read the bounding boxes
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    cls, x_center, y_center, width, height = map(float, line.strip().split())
                    valid_bbox = validate_and_clip_bbox([x_center * w, y_center * h, width * w, height * h], w, h)
                    if valid_bbox:
                        bboxes.append(valid_bbox)
                        class_labels.append(int(cls))

        if not bboxes:
            print(f"Skipping image {image_file} due to no valid bounding boxes.")
            continue

        # Apply Augmentations
        augmented = augmentations(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = augmented["image"]
        augmented_bboxes = augmented["bboxes"]
        augmented_class_labels = augmented["class_labels"]

        # 🔹 Fix Black Image Issue: Convert Back to uint8 before saving
        augmented_image = denormalize_image(augmented_image)

        # Save Augmented Image
        output_img_path = os.path.join(output_image_folder, image_file)
        cv2.imwrite(output_img_path, augmented_image)

        # Save Updated Labels
        output_label_path = os.path.join(output_label_folder, os.path.splitext(image_file)[0] + ".txt")
        with open(output_label_path, "w") as f:
            for bbox, cls in zip(augmented_bboxes, augmented_class_labels):
                x_center, y_center, width, height = bbox
                f.write(f"{cls} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        print(f"Saved Augmented Image: {output_img_path}")

# Apply Augmentation to Train and Validation Sets
dataset_dir = "/content/dataset/dataset"
advanced_augmentations(
    image_folder=f"{dataset_dir}/train/images",
    label_folder=f"{dataset_dir}/train/labels",
    output_image_folder=f"{augmented_dir}/train/images",
    output_label_folder=f"{augmented_dir}/train/labels"
)

advanced_augmentations(
    image_folder=f"{dataset_dir}/val/images",
    label_folder=f"{dataset_dir}/val/labels",
    output_image_folder=f"{augmented_dir}/val/images",
    output_label_folder=f"{augmented_dir}/val/labels"
)

print("✅ Augmentation completed successfully.")

In [ ]:
# Load the trained YOLOv8 model
model = YOLO("/content/drive/MyDrive/tomato_ripeness_classifier/yolov8_tomato_ripeness/weights/best.pt")

# Run evaluation on the validation set
metrics = model.val()

# Extract key metrics
precision = metrics.box.p  # Precision values per class
recall = metrics.box.r      # Recall values per class
map50 = metrics.box.map50  # mAP@50 values per class
map50_95 = metrics.box.map  # mAP@50-95 values per class
classes = metrics.names     # Class names


# Convert metrics to lists
class_labels = list(classes.values())  # Extract class names
num_classes = len(class_labels)

# Plot Precision & Recall
plt.figure(figsize=(10, 5))
plt.bar(class_labels, precision, color="blue", label="Precision")
plt.bar(class_labels, recall, color="red", alpha=0.7, label="Recall")
plt.xlabel("Classes")
plt.ylabel("Score")
plt.title("Precision & Recall per Class")
plt.legend()
plt.xticks(rotation=45)
plt.savefig("/content/precision_recall_plot.png")
plt.show()

# Plot mAP@50 and mAP@50-95
plt.figure(figsize=(10, 5))
plt.bar(class_labels, map50, color="green", label="mAP@50")
plt.bar(class_labels, map50_95, color="orange", alpha=0.7, label="mAP@50-95")
plt.xlabel("Classes")
plt.ylabel("Score")
plt.title("mAP Scores per Class")
plt.legend()
plt.xticks(rotation=45)
plt.savefig("/content/map_plot.png")
plt.show()

print("Evaluation Complete. Saved plots as images.")


In [ ]:
import os
import json
from PIL import Image

# Paths to your dataset
image_dir = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/images"
label_dir = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/labels"
output_json = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/annotations.json"

# Load class names from YAML
class_names = ["b_fully_ripened", "b_half_ripened", "b_green", "l_fully_ripened", "l_half_ripened", "l_green"]
class_mapping = {i: name for i, name in enumerate(class_names)}

# COCO JSON structure
coco_data = {
    "images": [],
    "annotations": [],
    "categories": [{"id": i, "name": name, "supercategory": "none"} for i, name in class_mapping.items()]
}

annotation_id = 0
for img_id, img_name in enumerate(os.listdir(image_dir)):
    if not img_name.endswith((".jpg", ".png", ".jpeg")):
        continue

    # Image details
    img_path = os.path.join(image_dir, img_name)
    img = Image.open(img_path)
    width, height = img.size

    coco_data["images"].append({
        "id": img_id,
        "file_name": img_name,
        "width": width,
        "height": height
    })

    # Corresponding YOLO label file
    label_file = os.path.join(label_dir, os.path.splitext(img_name)[0] + ".txt")
    if not os.path.exists(label_file):
        continue

    with open(label_file, "r") as f:
        for line in f.readlines():
            parts = line.strip().split()
            # Convert to int after converting to float to handle '3.0' like values
            class_id = int(float(parts[0]))
            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:])

            # Convert YOLO format (normalized) to COCO format (absolute pixel values)
            x_min = (x_center - bbox_width / 2) * width
            y_min = (y_center - bbox_height / 2) * height
            bbox_width *= width
            bbox_height *= height

            coco_data["annotations"].append({
                "id": annotation_id,
                "image_id": img_id,
                "category_id": class_id,
                "bbox": [x_min, y_min, bbox_width, bbox_height],
                "area": bbox_width * bbox_height,
                "iscrowd": 0
            })
            annotation_id += 1

# Save to COCO JSON file
with open(output_json, "w") as f:
    json.dump(coco_data, f, indent=4)

print(f"COCO annotations saved to {output_json}")

In [ ]:
import json
import cv2
import matplotlib.pyplot as plt
import random
import os

# Paths
image_dir = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/images"
annotation_file = "/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/annotations.json"

# Load COCO annotations
with open(annotation_file, "r") as f:
    coco_data = json.load(f)

# Get a random image from the dataset
random_image = random.choice(coco_data["images"])
image_path = os.path.join(image_dir, random_image["file_name"])

# Load image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Find corresponding annotations
image_id = random_image["id"]
bboxes = [ann["bbox"] for ann in coco_data["annotations"] if ann["image_id"] == image_id]
labels = [ann["category_id"] for ann in coco_data["annotations"] if ann["image_id"] == image_id]

# Draw bounding boxes
for bbox, label in zip(bboxes, labels):
    x, y, w, h = map(int, bbox)
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    label_name = next((cat["name"] for cat in coco_data["categories"] if cat["id"] == label), "Unknown")
    cv2.putText(image, label_name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Show image with bounding boxes
plt.figure(figsize=(8, 6))
plt.imshow(image)
plt.axis("off")
plt.title("COCO Annotations Visualization")
plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Load Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 7  # 6 classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.load_state_dict(torch.load("/content/drive/MyDrive/tomato_ripeness_classifier/fasterrcnn_tomato_model/model.pth"))
model.eval()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the transformation function
def transform_fn(image, target):
    # Resize the image to a fixed size
    image = transforms.Resize((800, 800))(image)  # Choose your desired size

    # Convert COCO annotations to the format expected by MeanAveragePrecision
    new_target = []
    for ann in target:
        new_target.append({
            'boxes': torch.tensor(ann['bbox'], dtype=torch.float32).unsqueeze(0), # Convert boxes to tensor
            'labels': torch.tensor([ann['category_id']], dtype=torch.int64) # Convert category_id to tensor
        })
    return F.to_tensor(image), new_target

# Load validation dataset (COCO format)
val_dataset = CocoDetection(
    root="/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/images",
    annFile="/content/drive/MyDrive/tomato_ripeness_classifier/dataset_augmented/val/annotations.json",
    transforms=transform_fn
)

# Define a custom collate function to handle variable-sized images
def collate_fn(batch):
    images, targets = zip(*batch)  # Unpack images and targets
    images = list(images)  # Convert to list for variable sizes
    targets = list(targets)  # Convert to list
    return images, targets

# Use DataLoader with the custom collate function
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

# Initialize metric
metric = MeanAveragePrecision().to(device)

# Run evaluation
for images, targets in val_loader:
    images = [img.to(device) for img in images]
    targets = [{k: v.to(device) for k, v in t[0].items()} for t in targets] # Move targets to device
    with torch.no_grad():
        preds = model(images)
    metric.update(preds, targets)

# Compute and display mAP results
map_metrics = metric.compute()
print(map_metrics)

# Visualize mAP per class
class_names = ["b_fully_ripened", "b_half_ripened", "b_green", "l_fully_ripened", "l_half_ripened", "l_green"]  # Your class names
map_per_class = map_metrics["map_per_class"]

# Check if map_per_class is available
if map_per_class is not None:
    map_values = map_per_class.cpu().numpy()  # Move to CPU and convert to NumPy

    plt.figure(figsize=(10, 5))
    plt.bar(class_names, map_values)
    plt.xlabel("Classes")
    plt.ylabel("mAP")
    plt.title("mAP per Class")
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for better readability
    plt.tight_layout()  # Adjust layout to prevent labels from being cut off
    plt.show()
else:
    print("mAP per class is not available in the metrics.")

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import functional as F

# Load the trained YOLOv8 model
from ultralytics import YOLO
yolo_model = YOLO("/content/drive/MyDrive/tomato_ripeness_classifier/yolov8_tomato_ripeness/weights/best.pt")

# Load the trained Faster R-CNN model
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

frcnn_model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
num_classes = 7  # 6 classes + background
in_features = frcnn_model.roi_heads.box_predictor.cls_score.in_features
frcnn_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
frcnn_model.load_state_dict(torch.load("/content/drive/MyDrive/tomato_ripeness_classifier/fasterrcnn_tomato_model/model.pth"))
frcnn_model.eval()

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
frcnn_model.to(device)

# Class names from your dataset
class_names = ["b_fully_ripened", "b_half_ripened", "b_green", "l_fully_ripened", "l_half_ripened", "l_green"]

# Get a Random Image from the Dataset

image_folder = "/content/dataset/dataset/val/images"
random_image_name = random.choice(os.listdir(image_folder))
img_path = os.path.join(image_folder, random_image_name)

# Load the image
image = Image.open(img_path).convert("RGB")
print(f"Randomly Selected Image: {random_image_name}")

# Function to draw bounding boxes
def draw_boxes(image, boxes, labels, scores, threshold=0.5):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for box, label, score in zip(boxes, labels, scores):
        if score < threshold:
            continue
        x1, y1, x2, y2 = box
        label_text = f"{class_names[label]}: {score:.2f}"

        # Draw rectangle
        draw.rectangle([(x1, y1), (x2, y2)], outline="red", width=3)

        # Draw label
        text_size = draw.textbbox((0, 0), label_text, font=font)
        draw.rectangle([x1, y1 - text_size[3], x1 + text_size[2], y1], fill="red")
        draw.text((x1, y1 - text_size[3]), label_text, fill="white", font=font)

    return image

# YOLOv8 Inference

yolo_preds = yolo_model(img_path, conf=0.5)

# Get YOLOv8 detections
yolo_classes = [class_names[int(box.cls)] for box in yolo_preds[0].boxes]
yolo_counts = dict(Counter(yolo_classes))  # Count occurrences of each class

print("\n📌 YOLOv8 Detections:")
for cls, count in yolo_counts.items():
    print(f"🔹 {cls}: {count}")

# Show YOLO detections
yolo_preds[0].show()

# Faster R-CNN Inference

image_tensor = F.to_tensor(image).unsqueeze(0).to(device)

# Perform inference
with torch.no_grad():
    frcnn_preds = frcnn_model(image_tensor)

# Process predictions
boxes = frcnn_preds[0]['boxes'].cpu().numpy()  # Bounding boxes
labels = frcnn_preds[0]['labels'].cpu().numpy() - 1  # Class labels (adjust for 0-based indexing)
scores = frcnn_preds[0]['scores'].cpu().numpy()  # Confidence scores

# Filter predictions (Threshold = 0.5)
valid_indices = scores > 0.5
boxes = boxes[valid_indices]
labels = labels[valid_indices]
scores = scores[valid_indices]

# Count class detections for Faster R-CNN
frcnn_classes = [class_names[label] for label in labels]
frcnn_counts = dict(Counter(frcnn_classes))  # Count occurrences of each class

print("\n📌 Faster R-CNN Detections:")
for cls, count in frcnn_counts.items():
    print(f"🔹 {cls}: {count}")

# Draw Faster R-CNN detections on the image
frcnn_image = draw_boxes(image.copy(), boxes, labels, scores)

# Show the Faster R-CNN results
plt.figure(figsize=(10, 6))
plt.imshow(frcnn_image)
plt.axis("off")
plt.title("Faster R-CNN Predictions")
plt.show()
